In [1]:
import numpy as np 
from scipy.integrate import odeint
import os, sys, warnings
from pathlib import Path
from os.path import dirname, realpath
script_dir = Path(dirname(realpath('.')))
module_dir = str(script_dir)
sys.path.insert(0, module_dir + '/modules')
import utility as ut
import surrogate_nn as srnn
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch import nn
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, TensorDataset
warnings.filterwarnings('ignore')

In [2]:
DTYPE = 'float64'
L63_data_path = '../data/L63-trajectories'
save_folder='../data/adaptive-rate'
log_interval = 100
milestones = [10*2**n for n in range(15)]
learning_rate = 1e-3
drop = 0.7
steps = int(2e5)
save_interval = 100
N = 2000
L0 = 0.4
L1 = 3.5
beta = 4e-5
min_change = (1.-0.1)**(1./log_interval) - 1.
train = np.load(f'{L63_data_path}/train.npy').astype(DTYPE)
test = np.load(f'{L63_data_path}/test.npy')[:, :, :1000].astype(DTYPE)

model = srnn.SurrogateModel_NN(3, 100, name='nn', save_folder=save_folder)
model.learn(train[:, :N], steps, 1e-3, batch_size='GD', log_interval=100, save_interval=1000,\
           max_delta=0.1, min_change=min_change, constant_rate=False, update_frequency=1000, mode='BS')
tau_f_rmse, tau_f_se, rmse, se = model.compute_tau_f(test[:100], error_threshold=0.05)
tau_f_1 = tau_f_se.mean()
tau_f_1

step: 0 loss: 1531508.297537 time: 0.0023 lr: 0.001000,  change: nan%
step: 100 loss: 733710.908441 time: 0.1183 lr: 0.001000,  change: -52.09%
step: 200 loss: 367854.218776 time: 0.2225 lr: 0.001000,  change: -49.86%
step: 300 loss: 226793.692970 time: 0.3310 lr: 0.001000,  change: -38.35%
step: 400 loss: 160936.946931 time: 0.4380 lr: 0.001000,  change: -29.04%
step: 500 loss: 120408.687692 time: 0.5441 lr: 0.001000,  change: -25.18%
step: 600 loss: 90446.625963 time: 0.6499 lr: 0.001000,  change: -24.88%
step: 700 loss: 65758.660904 time: 0.7560 lr: 0.001000,  change: -27.30%
step: 800 loss: 47686.444021 time: 0.8604 lr: 0.001000,  change: -27.48%
step: 900 loss: 35037.765220 time: 0.9653 lr: 0.001000,  change: -26.52%
step: 1000 loss: 26078.665900 time: 1.0728 lr: 0.001000,  change: -25.57%
step: 1100 loss: 20040.877844 time: 1.1812 lr: 0.001000,  change: -23.15%
step: 1200 loss: 16021.547294 time: 1.2879 lr: 0.001000,  change: -20.06%
step: 1300 loss: 13163.115904 time: 1.3946 lr:

1.245426

In [3]:
model.net.state_dict()['2.weight'].requires_grad

False

In [8]:
(beta*torch.sum(model.net[2].weight**2)).requires_grad

True